In [1]:
!pip install opencv-python-headless tensorflow keras matplotlib ultralytics scikit-learn
!pip install deepface opencv-python-headless ultralytics

# Download OpenCV DNN face detector model files
!wget -O deploy.prototxt https://raw.githubusercontent.com/opencv/opencv/master/samples/dnn/face_detector/deploy.prototxt
!wget -O res10_300x300_ssd_iter_140000.caffemodel https://github.com/keyurr2/face-detection/raw/master/res10_300x300_ssd_iter_140000.caffemodel


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.3/128.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.9/115.9 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.3 MB/s eta 0:00:00
--2025-09-03 09:08:13--  https://raw.githubusercontent.com/opencv/opencv/master/samples/dnn/face_detector/deploy.prototxt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 28104 (27K) [text/plain]
Saving to: ‘deploy.prototxt’

deploy.prototxt     100%[===================>]  27.45K  --.-KB/s    in 0.0

In [2]:

import cv2
import numpy as np
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
from google.colab import files
from ultralytics import YOLO
from PIL import Image
import os
from datetime import datetime, timedelta
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
from deepface import DeepFace

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
25-09-03 09:08:49 - Directory /root/.deepface has been created
25-09-03 09:08:49 - Directory /root/.deepface/weights has been created


In [ ]:

uploaded_images = files.upload()

known_face_embeddings = []
known_face_names = []

for filename in uploaded_images.keys():
    name = filename.split('.')[0]
    try:
        embedding_obj = DeepFace.represent(filename, model_name='Facenet', enforce_detection=False)
        embedding = embedding_obj[0]["embedding"]
        known_face_embeddings.append(embedding)
        known_face_names.append(name)
        print(f"✅ Processed known face: {name}")
    except Exception as e:
        print(f"❌ Error processing {filename}: {e}")

In [ ]:
#Upload video
uploaded_video = files.upload()
video_path = list(uploaded_video.keys())[0]

# Load video and models
cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
model = YOLO('yolov8n.pt')

# Load face detector
face_detector = cv2.dnn.readNetFromCaffe('deploy.prototxt', 'res10_300x300_ssd_iter_140000.caffemodel')


In [3]:
from datetime import datetime

entry_line = [(100, 200), (500, 200)]
entry_count = 0
restricted_zone = [(300, 100), (500, 100), (500, 300), (300, 300)]
log = []
recognized_faces = []
restricted_hours = [(22, 24), (0, 6)]

def is_crossing_line(y_center, line_y=200):
    return line_y - 5 < y_center < line_y + 5

def is_inside_zone(x1, y1, x2, y2, zone):
    cx = (x1 + x2) // 2
    cy = (y1 + y2) // 2
    return cv2.pointPolygonTest(np.array(zone, np.int32), (cx, cy), False) >= 0

def is_restricted_time():
    now = datetime.now().hour
    return any(start <= now < end for start, end in restricted_hours)

def log_event(event_type, frame_num, timestamp, person_id="Unknown"):
    log.append({
        "event": event_type,
        "frame": frame_num,
        "timestamp": timestamp,
        "person_id": person_id
    })



In [ ]:
rame_num = 0
recognized_faces = []
entry_count = 0
log = []

def log_event(event_type, frame_num, timestamp, person_id="Unknown"):
    log.append({
        "event": event_type,
        "frame": frame_num,
        "timestamp": timestamp,
        "person_id": person_id
    })

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_num += 1
    timestamp = str(timedelta(seconds=frame_num / fps))
    h, w = frame.shape[:2]

    # Face detection using OpenCV DNN
    blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300), [104, 117, 123], False, False)
    face_detector.setInput(blob)
    detections = face_detector.forward()

    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.5:
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            x1, y1, x2, y2 = box.astype("int")
            x1, y1 = max(0, x1), max(0, y1)
            x2, y2 = min(w, x2), min(h, y2)

            face_img = frame[y1:y2, x1:x2]
            name = "Unknown"

            try:
                embedding_obj = DeepFace.represent(face_img, model_name='Facenet', enforce_detection=False)
                embedding = embedding_obj[0]["embedding"]

                if known_face_embeddings:
                    sims = cosine_similarity([embedding], known_face_embeddings)[0]
                    best_match = np.argmax(sims)
                    best_score = sims[best_match]
                    print(f"Frame {frame_num} similarity scores: {sims}")

                    if best_score > 0.9:
                        name = known_face_names[best_match]

                if name == "Unknown":
                    log_event("Unknown Face Detected", frame_num, timestamp)

                recognized_faces.append({"name": name, "frame": frame_num, "timestamp": timestamp})
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, name, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2)

            except Exception as e:
                print(f"Error processing face: {e}")

    # Optional: Display entry count
    cv2.putText(frame, f"Entries: {entry_count}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    cv2_imshow(frame)

cap.release()

In [ ]:
known_faces_count = sum(1 for face in recognized_faces if face['name'] != 'Unknown')
unknown_faces_count = sum(1 for face in recognized_faces if face['name'] == 'Unknown')
timestamps = [face['timestamp'] for face in recognized_faces if face['name'] == 'Unknown']

print(f"Detected {known_faces_count} known face(s) and {unknown_faces_count} unknown face(s) in the video.")
print("Timestamps when unknown faces were detected:", timestamps)

log.append({"event": "Summary", "frame": "-", "timestamp": "-", "person_id": f"{known_faces_count} known, {unknown_faces_count} unknown"})

pd.DataFrame(log).to_csv("event_log.csv", index=False)
files.download("event_log.csv")